# Main Fragmented Landscapes Pipeline

A big pipeline to run all the simulations in a single, parrelised way

The intention here is that we run across 5 different sizes: 50x50, 500x500 and 5000x5000

In each case, we run across a selection of real landscapes, random landscapes and contrived landscapes, for 10, 25 and 50% habitat cover.

The real landscapes are created by finding real areas of habitat in the amazon which are close to the desired habitat cover across the desired area, and generating maps with precisely the desired habitat cover from there.

The random landscapes area created by randomly adding pixels until we reach our desired landscape cover.

The contrived landscapes are created using the FragmentedLandscape class from pycoalescence, which generates evenly-spaced, evenly-sized fragments across a landscape of the desired size, with the specified total habitat coverage.

In [6]:
import os
from fragment_generation import extract_landscape
import numpy as np
from pycoalescence import Map
from random import randint
from osgeo import gdal

In [8]:
def write_to_raster(grid: np.array, file):
	"""
	Writes numpy array to desired file
	:param arr: the numpy array to write to file
	:param file: file to output to
	"""
	geotransform = (0, 1, 0, 0, 0, -1)
	# Stupid attempt to fix the no data issue for writing out numpy arrays
	for i in range(max(grid.shape)):
		if grid[i,i] == 0:
			grid[i,i] == 0
	output_raster = gdal.GetDriverByName('GTiff').Create(file,
														 grid.shape[0], grid.shape[1], 1,
														 gdal.GDT_Byte)
	output_raster.SetGeoTransform(geotransform)
	out_band = output_raster.GetRasterBand(1)
	out_band.WriteArray(grid)
	out_band.FlushCache()
	out_band.SetNoDataValue(-99)
	del output_raster

In [4]:
# Okay, so first let's generate the real landscapes.
out_dir = "../../Data/Maps/FragmentMaps/CentralAmericaExtractions/"
if not os.path.exists(out_dir):
	os.mkdir(out_dir)
global ca_forest
ca_forest = None
for size in [50, 500]:
	for cover in [0.1, 0.25, 0.5]:
		print(cover)
		for file_no in range(100):
			file_name = os.path.join(out_dir, "map_{}_{}_{}.tif".format(size, cover, file_no))
			if not os.path.exists(file_name):
				extract_landscape(desired_cover=cover, size=size, number=100, output_dir=out_dir)
				break

0.1
0.25
0.5
0.1
0.25
0.5


In [9]:
# Now stitch random 500x500 maps together for the 5000x5000 ones.
map_list_500_01 = [x for x in os.listdir(out_dir) if "500_0.1" in x]
map_list_500_02 = [x for x in os.listdir(out_dir) if "500_0.2" in x]
all_maps = {0.1: map_list_500_01, 0.2: map_list_500_02}
# create 10 random maps for each percentage cover
for pc in [0.1, 0.2]:
	map_list = all_maps[pc]
	for i in range(10):
		tmp = np.empty(shape=[5000, 5000])
		for x in range(0, 5000, 500):
			for y in range(0, 5000, 500):
				random_map = map_list[randint(0, len(map_list) - 1)]
				m = Map(file=os.path.join(out_dir, random_map))
				tmp[x:x+500, y:y+500] = m.get_subset(0, 0, 500, 500)
		out_file = os.path.join(out_dir, "map_5000_{}_{}.tif".format(pc, i))
		write_to_raster(tmp, out_file)